# **LEMATIZACIÓN DE DATOS TF IDF**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import spacy
import unicodedata
import re
from tqdm import tqdm
import os

In [5]:
# ------------------------------------------------------------
# 1️⃣ CONFIGURACIÓN GENERAL
# ------------------------------------------------------------
spacy.prefer_gpu()  # Usa GPU si está disponible

# Rutas de trabajo
IN_PATH = "/content/drive/MyDrive/classification_coding_open_ended_occupational_responses_ENAHO/CLEAN_DATA/BASE_LIMPIA_VF.parquet"
OUT_PATH = "/content/drive/MyDrive/classification_coding_open_ended_occupational_responses_ENAHO/TF-IDF/BASE_LEMATIZADA.parquet"

In [6]:
# ------------------------------------------------------------
# 2️⃣ PREPROCESAMIENTO LIGERO (antes de lematizar)
# ------------------------------------------------------------
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    # Minúsculas
    texto = texto.lower()
    # Quitar tildes (pero conservar ñ)
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn' or c == 'ñ'
    )
    # Eliminar puntuación, caracteres especiales y números
    texto = re.sub(r"[^a-zñ\s]", " ", texto)
    # Normalizar espacios
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

## Cargar base de datos

In [7]:
# ------------------------------------------------------------
# 3️⃣ CARGA DE DATOS
# ------------------------------------------------------------
df = pd.read_parquet(IN_PATH)
print(f"✅ Base cargada: {df.shape[0]:,} filas")

# Crear texto concatenado
df["texto_concat"] = (
    df["txt505_clean"].fillna("") + " " +
    df["txt505b_clean"].fillna("") + " " +
    df["txt506_clean"].fillna("")
).apply(limpiar_texto)

✅ Base cargada: 316,022 filas


In [9]:
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 25.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
# ------------------------------------------------------------
# 4️⃣ LEMATIZACIÓN CON spaCy (batch + multiproceso)
# ------------------------------------------------------------
print("🧠 Cargando modelo spaCy...")
nlp = spacy.load("es_core_news_md", disable=["ner", "parser"])
nlp.max_length = 3_000_000

# Lista de acrónimos laborales válidos que NO deben eliminarse por cortos
ACRONIMOS_VALIDOS = {"dj", "qa", "ti", "rrhh", "it", "ux", "ui", "hr", "tv"}

# Stopwords personalizadas (añade o quita según tu contexto)
STOPWORDS = nlp.Defaults.stop_words.union({
    "trabajo", "labor", "oficio", "puesto", "cargo"
})

def lematizar_texto(texto):
    doc = nlp(texto)
    tokens = []
    for t in doc:
        if (
            not t.is_punct
            and not t.is_space
            and t.lemma_ not in STOPWORDS
        ):
            # Mantener acrónimos cortos relevantes
            if len(t.lemma_) <= 2 and t.lemma_ not in ACRONIMOS_VALIDOS:
                continue
            tokens.append(t.lemma_)
    return " ".join(tokens)

# Procesamiento por lotes con nlp.pipe
def lemmatize_pipe(texts, batch_size=1000):
    for doc in nlp.pipe(texts, batch_size=batch_size, n_process=os.cpu_count()):
        tokens = []
        for t in doc:
            if not t.is_punct and not t.is_space and t.lemma_ not in STOPWORDS:
                if len(t.lemma_) <= 2 and t.lemma_ not in ACRONIMOS_VALIDOS:
                    continue
                tokens.append(t.lemma_)
        yield " ".join(tokens)

print("⚙️ Lematizando texto (esto puede tardar unos minutos)...")
df["texto_lematizado"] = list(lemmatize_pipe(df["texto_concat"].astype(str)))


🧠 Cargando modelo spaCy...
⚙️ Lematizando texto (esto puede tardar unos minutos)...


In [12]:
# ------------------------------------------------------------
# 🔍 Verificación previa a guardar
# ------------------------------------------------------------

# Muestra aleatoria de 10 filas para revisar resultados
muestra = df.sample(10, random_state=2025)[
    ["txt505_clean", "txt505b_clean", "txt506_clean", "texto_concat", "texto_lematizado"]
]

# Mostrar en formato legible
pd.set_option("display.max_colwidth", 150)
print("🔎 Ejemplo de lematización:\n")
display(muestra)

🔎 Ejemplo de lematización:



,txt505_clean,txt505b_clean,txt506_clean,texto_concat,texto_lematizado
144243,vendedora de zapatillas juguetes ropas interiores,atender cliente vender cobrar,venta por menor de zapatillas lenceria juguetes en tienda,vendedora de zapatillas juguetes ropas interiores atender cliente vender cobrar venta por menor de zapatillas lenceria juguetes en tienda,vendedora zapatilla juguet ropa interior atender cliente vender cobrar venta menor zapatilla lencerio juguetes tienda
37187,productor avícola,dar agua alimentos a los animales,crianza de aves de corral,productor avicola dar agua alimentos a los animales crianza de aves de corral,productor avicolo agua alimento animal crianza ave corral
81076,peón agricola,cultivar maíz papa,cultivo de maíz papa trigo crianza vacuno aves de corral cuy,peon agricola cultivar maiz papa cultivo de maiz papa trigo crianza vacuno aves de corral cuy,peon agricola cultivar maiz papa cultivo maiz papa trigo crianza vacuno ave corral cuy
6147,pron agricola,coger coca,cultivo de coca,pron agricola coger coca cultivo de coca,pron agricola coger coca cultivo coca
101584,vendedor de abarrotes,vender abarrotes,venta de abarrotes al por menor en vivienda,vendedor de abarrotes vender abarrotes venta de abarrotes al por menor en vivienda,vendedor abarrot vender abarrot venta abarrot menor vivienda
182594,trabajadora del hogar,cocinar picar carne verduras lavar plato olla barrer trapear,vivienda particular,trabajadora del hogar cocinar picar carne verduras lavar plato olla barrer trapear vivienda particular,trabajadora hogar cocinar picar carne verdura lavar plato ollo barrer trapear vivienda particular
45083,jefe de registro civil en cargada del area de renta,inscribir partidas hacer rectificación de partida cobrar alquiler de m,municipalidad distrital,jefe de registro civil en cargada del area de renta inscribir partidas hacer rectificacion de partida cobrar alquiler de m municipalidad distrital,jefe registro civil cargado area renta inscribir partida rectificacion partida cobrar alquiler municipalidad distrital
211475,productor agropecuario,pastar vacuno aporcar papa,crianza de vacuno cuy cultivo de papa quinua haba maíz,productor agropecuario pastar vacuno aporcar papa crianza de vacuno cuy cultivo de papa quinua haba maiz,productor agropecuario pastar vacuno aporcar papa crianza vacuno cuy cultivo papa quinua har maiz
50164,medico general,evaluar diagnosticar paciente,hospital essalud,medico general evaluar diagnosticar paciente hospital essalud,medico general evaluar diagnosticar paciente hospital essalud
30390,ayudante de productor agropecuario,cortar avena amarrar vacuno ovino,cultivo de papa avena cebada crianza de ganado vacuno ovino ave de cor,ayudante de productor agropecuario cortar avena amarrar vacuno ovino cultivo de papa avena cebada crianza de ganado vacuno ovino ave de cor,ayudante productor agropecuario cortar avena amarrar vacuno ovino cultivo papa avén cebado crianza ganado vacuno ovino ave cor


In [11]:
# ------------------------------------------------------------
# 5️⃣ SALIDA FINAL (mantener todas las columnas originales)
# ------------------------------------------------------------

# Solo añadimos las columnas nuevas al DataFrame original
# (ya están dentro de df, no hace falta crear df_out reducido)
OUT_PATH = "/content/drive/MyDrive/classification_coding_open_ended_occupational_responses_ENAHO/TF-IDF/BASE_LEMATIZADA.parquet"

# Guardar TODAS las columnas + las nuevas
df.to_parquet(OUT_PATH, index=False)

print(f"💾 Archivo completo guardado en: {OUT_PATH}")
print(f"Columnas finales: {len(df.columns)}")
print("✅ Lematización y limpieza completadas con éxito.")


💾 Archivo completo guardado en: /content/drive/MyDrive/classification_coding_open_ended_occupational_responses_ENAHO/TF-IDF/BASE_LEMATIZADA.parquet
Columnas finales: 29
✅ Lematización y limpieza completadas con éxito.
